In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from subprocess import check_output
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import xgboost as xgb
from sklearn import model_selection
import scipy.stats as stats
import gc

/home/jaromir/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [32]:
train = pd.read_csv('~/data/bimbo/train.csv', nrows=20000)
#train = train.sample(n=20000)
test = pd.read_csv('~/data/bimbo/test.csv', nrows=5000)
#test = test.sample(n=2000)


In [33]:
train.describe()

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil
count,20000.0,20000.000000,20000.00000,20000.000000,2.000000e+04,20000.00000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000
mean,3.0,1110.599300,3.55765,1941.095150,1.266374e+06,11497.17740,9.657500,118.425184,0.206350,1.914294,9.616250
std,0.0,0.490053,3.20565,1137.059325,1.546607e+06,16038.13817,38.728387,334.852973,8.854608,37.393328,38.731592
min,3.0,1110.000000,1.00000,1001.000000,1.407500e+04,73.00000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.0,1110.000000,1.00000,1010.000000,6.194500e+04,1146.00000,2.000000,28.800000,0.000000,0.000000,2.000000
50%,3.0,1111.000000,1.00000,1020.000000,8.128110e+05,2233.00000,5.000000,54.000000,0.000000,0.000000,5.000000
75%,3.0,1111.000000,7.00000,3309.000000,1.975061e+06,30574.00000,9.000000,113.160000,0.000000,0.000000,9.000000
max,3.0,1111.000000,11.00000,3504.000000,9.678222e+06,49185.00000,2000.000000,15561.000000,1008.000000,3030.880000,2000.000000


In [3]:
train.head(n=20)

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil
0,3,1110,7,3301,15766,1212,3,25.14,0,0.0,3
1,3,1110,7,3301,15766,1216,4,33.52,0,0.0,4
2,3,1110,7,3301,15766,1238,4,39.32,0,0.0,4
3,3,1110,7,3301,15766,1240,4,33.52,0,0.0,4
4,3,1110,7,3301,15766,1242,3,22.92,0,0.0,3
5,3,1110,7,3301,15766,1250,5,38.20,0,0.0,5
6,3,1110,7,3301,15766,1309,3,20.28,0,0.0,3
7,3,1110,7,3301,15766,3894,6,56.10,0,0.0,6
8,3,1110,7,3301,15766,4085,4,24.60,0,0.0,4
9,3,1110,7,3301,15766,5310,6,31.68,0,0.0,6


In [30]:
test.describe()

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,tst
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5.000000e+03,5000.000000,5000.0
mean,2499.500000,10.489000,2463.321800,1.371200,2129.290600,1.837505e+06,22247.839600,1.0
std,1443.520003,0.499929,3876.730237,1.462272,1494.829478,1.864852e+06,18685.732455,0.0
min,0.000000,10.000000,1110.000000,1.000000,2.000000,2.755000e+03,72.000000,1.0
25%,1249.750000,10.000000,1311.000000,1.000000,1160.000000,3.605088e+05,1250.000000,1.0
50%,2499.500000,10.000000,1612.000000,1.000000,1293.500000,1.229744e+06,31506.500000,1.0
75%,3749.250000,11.000000,2037.000000,1.000000,2803.000000,2.385908e+06,40930.000000,1.0
max,4999.000000,11.000000,25759.000000,11.000000,9328.000000,9.838259e+06,49973.000000,1.0


In [4]:
test.head()

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID
0,0,11,4037,1,2209,4639078,35305
1,1,11,2237,1,1226,4705135,1238
2,2,10,2045,1,2831,4549769,32940
3,3,11,1227,1,4448,4717855,43066
4,4,11,1219,1,1130,966351,1277


In [5]:
train['target'] = train['Demanda_uni_equil']
train.drop(['Demanda_uni_equil'],axis=1, inplace = True)

train['tst'] = 0
test['tst'] = 1


In [6]:
data = pd.concat([train,test], axis=0, copy=True)


In [7]:
data.head(n=20)

,Agencia_ID,Canal_ID,Cliente_ID,Dev_proxima,Dev_uni_proxima,Producto_ID,Ruta_SAK,Semana,Venta_hoy,Venta_uni_hoy,id,target,tst
0,1110,7,15766,0.0,0.0,1212,3301,3,25.14,3.0,NaN,3.0,0
1,1110,7,15766,0.0,0.0,1216,3301,3,33.52,4.0,NaN,4.0,0
2,1110,7,15766,0.0,0.0,1238,3301,3,39.32,4.0,NaN,4.0,0
3,1110,7,15766,0.0,0.0,1240,3301,3,33.52,4.0,NaN,4.0,0
4,1110,7,15766,0.0,0.0,1242,3301,3,22.92,3.0,NaN,3.0,0
5,1110,7,15766,0.0,0.0,1250,3301,3,38.20,5.0,NaN,5.0,0
6,1110,7,15766,0.0,0.0,1309,3301,3,20.28,3.0,NaN,3.0,0
7,1110,7,15766,0.0,0.0,3894,3301,3,56.10,6.0,NaN,6.0,0
8,1110,7,15766,0.0,0.0,4085,3301,3,24.60,4.0,NaN,4.0,0
9,1110,7,15766,0.0,0.0,5310,3301,3,31.68,6.0,NaN,6.0,0


In [8]:
for i in range(1,6):
    lag = 'Lag' + str(i)
    print('Lag:',lag)
    
    data1 = data[['Semana','Cliente_ID','Producto_ID','target']]
    data1.loc[:,'Semana'] = data1['Semana'] +i
    data1 = data1.groupby(['Semana','Cliente_ID','Producto_ID']).mean()
    data1 = data1.reset_index()
    data1.rename(columns={'target': lag}, inplace=True)
    data = pd.merge(data,data1,
                    how='left',
                    left_on=['Semana','Cliente_ID','Producto_ID'], 
                    right_on=['Semana','Cliente_ID','Producto_ID'],
                    left_index=False, right_index=False, sort=True,
                    suffixes=('_x', '_y'), copy=False, )
    del data1
    gc.collect()


/home/jaromir/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Lag: Lag1
Lag: Lag2
Lag: Lag3
Lag: Lag4
Lag: Lag5


In [9]:
data['TotalLags'] = data['Lag1'] + data['Lag2']+ data['Lag3']+ data['Lag4']+ data['Lag5']
data.head()

,Agencia_ID,Canal_ID,Cliente_ID,Dev_proxima,Dev_uni_proxima,Producto_ID,Ruta_SAK,Semana,Venta_hoy,Venta_uni_hoy,id,target,tst,Lag1,Lag2,Lag3,Lag4,Lag5,TotalLags
0,1111,1,14075,0.0,0.0,1109,1019,3,105.07,7.0,NaN,7.0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,1111,1,14075,0.0,0.0,1125,1019,3,38.40,4.0,NaN,4.0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,1111,1,14075,0.0,0.0,1129,1019,3,35.20,2.0,NaN,2.0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,1111,1,14075,0.0,0.0,1146,1019,3,42.78,2.0,NaN,2.0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,1111,1,14075,0.0,0.0,1182,1019,3,55.76,4.0,NaN,4.0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
data=data[data['Semana']>8]  # NOW I WORK WITH WEEKS 9,10,11
data.head()

,Agencia_ID,Canal_ID,Cliente_ID,Dev_proxima,Dev_uni_proxima,Producto_ID,Ruta_SAK,Semana,Venta_hoy,Venta_uni_hoy,id,target,tst,Lag1,Lag2,Lag3,Lag4,Lag5,TotalLags
20000,1243,1,3196,NaN,NaN,43058,2102,10,NaN,NaN,1114.0,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
20001,1227,1,3597,NaN,NaN,1240,1269,10,NaN,NaN,2108.0,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
20002,2012,1,3633,NaN,NaN,43064,2006,10,NaN,NaN,1208.0,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
20003,2012,1,3844,NaN,NaN,4245,5014,10,NaN,NaN,1809.0,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
20004,2012,1,3987,NaN,NaN,4767,1224,10,NaN,NaN,3972.0,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
data.groupby(['Agencia_ID','Semana']).count().head()

Canal_ID  Cliente_ID  Dev_proxima  Dev_uni_proxima  \
Agencia_ID Semana                                                       
1110       10             3           3            0                0   
           11             2           2            0                0   
1111       10            14          14            0                0   
           11            12          12            0                0   
1112       10            12          12            0                0   

                   Producto_ID  Ruta_SAK  Venta_hoy  Venta_uni_hoy  id  \
Agencia_ID Semana                                                        
1110       10                3         3          0              0   3   
           11                2         2          0              0   2   
1111       10               14        14          0              0  14   
           11               12        12          0              0  12   
1112       10               12        12          0              0  12   

                   target  tst  Lag1  Lag2  Lag3  Lag4  Lag5  TotalLags  
Agencia_ID Semana                                                        
1110       10           0    3     0     0     0     0     0          0  
           11           0    2     0     0     0     0     0          0  
1111       10           0   14     0     0     0     0     0          0  
           11           0   12     0     0     0     0     0          0  
1112       10           0   12     0     0     0     0     0          0

In [15]:
pd.DataFrame(data.groupby(['Agencia_ID','Semana']).count()).head()

Canal_ID  Cliente_ID  Dev_proxima  Dev_uni_proxima  \
Agencia_ID Semana                                                       
1110       10             3           3            0                0   
           11             2           2            0                0   
1111       10            14          14            0                0   
           11            12          12            0                0   
1112       10            12          12            0                0   

                   Producto_ID  Ruta_SAK  Venta_hoy  Venta_uni_hoy  id  \
Agencia_ID Semana                                                        
1110       10                3         3          0              0   3   
           11                2         2          0              0   2   
1111       10               14        14          0              0  14   
           11               12        12          0              0  12   
1112       10               12        12          0              0  12   

                   target  tst  Lag1  Lag2  Lag3  Lag4  Lag5  TotalLags  
Agencia_ID Semana                                                        
1110       10           0    3     0     0     0     0     0          0  
           11           0    2     0     0     0     0     0          0  
1111       10           0   14     0     0     0     0     0          0  
           11           0   12     0     0     0     0     0          0  
1112       10           0   12     0     0     0     0     0          0

In [22]:
data.groupby(['Agencia_ID','Semana'])['target'].describe()

count  mean  std  min  25%  50%  75%  max
Agencia_ID Semana                                           
1110       10        3.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
           11        2.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
1111       10       14.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
           11       12.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
1112       10       12.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
           11        7.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
1113       10        8.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
           11        3.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
1114       10        1.0   0.0  NaN  0.0  0.0  0.0  0.0  0.0
           11        2.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
1116       10       25.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
           11       19.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
1117       10       24.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
           11       16.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
1118       10        8.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
           11       11.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
1119       10       10.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
           11       11.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
1120       10       25.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
           11       20.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
1121       10       25.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
           11       17.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
1122       10       21.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
           11       11.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
1123       10       26.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
           11       20.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
1124       10       17.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
           11       17.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
1126       10       23.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
           11       30.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
...                  ...   ...  ...  ...  ...  ...  ...  ...
4064       11        2.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
4066       10        1.0   0.0  NaN  0.0  0.0  0.0  0.0  0.0
4078       10        1.0   0.0  NaN  0.0  0.0  0.0  0.0  0.0
4086       10        1.0   0.0  NaN  0.0  0.0  0.0  0.0  0.0
22090      10       12.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
           11       11.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
22187      10       10.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
           11       10.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
22362      10       14.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
           11        6.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
22414      10        3.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
           11        7.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
22560      10        6.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
           11       10.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
23669      10       10.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
           11        9.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
23719      10        7.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
           11       11.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
23879      10        3.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
           11        2.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
23899      10        4.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
           11        5.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
24049      10        2.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
           11        2.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
24539      11        1.0   0.0  NaN  0.0  0.0  0.0  0.0  0.0
24669      10        2.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
           11        4.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
25699      11        2.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
25759      10        5.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0
           11        6.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0

[689 rows x 8 columns]

In [13]:
data.groupby(['Agencia_ID','Semana'])['target'].count().head()

Agencia_ID  Semana
1110        10        0
            11        0
1111        10        0
            11        0
1112        10        0
Name: target, dtype: int64

In [26]:
nAgencia = pd.DataFrame(data.groupby(['Agencia_ID','Semana'])['target'].count())
nAgencia.head()

target
Agencia_ID Semana        
1110       10           3
           11           2
1111       10          14
           11          12
1112       10          12

In [27]:
nAgencia = nAgencia.reset_index()
nAgencia.rename(columns={'target': 'nAgencia'}, inplace=True)
nAgencia.head()

,Agencia_ID,Semana,nAgencia
0,1110,10,3
1,1110,11,2
2,1111,10,14
3,1111,11,12
4,1112,10,12


In [28]:
nAgencia.groupby(['Agencia_ID'])['nAgencia'].describe()

,count,mean,std,min,25%,50%,75%,max
Agencia_ID,,,,,,,,
1110,2.0,2.5,0.707107,2.0,2.25,2.5,2.75,3.0
1111,2.0,13.0,1.414214,12.0,12.50,13.0,13.50,14.0
1112,2.0,9.5,3.535534,7.0,8.25,9.5,10.75,12.0
1113,2.0,5.5,3.535534,3.0,4.25,5.5,6.75,8.0
1114,2.0,1.5,0.707107,1.0,1.25,1.5,1.75,2.0
1116,2.0,22.0,4.242641,19.0,20.50,22.0,23.50,25.0
1117,2.0,20.0,5.656854,16.0,18.00,20.0,22.00,24.0
1118,2.0,9.5,2.121320,8.0,8.75,9.5,10.25,11.0
1119,2.0,10.5,0.707107,10.0,10.25,10.5,10.75,11.0


In [17]:
nAgencia = pd.DataFrame(nAgencia.groupby(['Agencia_ID'])['nAgencia'].mean())
nAgencia = nAgencia.reset_index()
nAgencia.head()

,Agencia_ID,nAgencia
0,1110,0
1,1111,0
2,1112,0
3,1113,0
4,1114,0


In [18]:
data = pd.merge(data, nAgencia, 
                            how='left',
                            left_on=['Agencia_ID'], 
                            right_on=['Agencia_ID'],
                            left_index=False, right_index=False, sort=True,
                            suffixes=('_x', '_y'), copy=False) 

del nAgencia
gc.collect()
print('merge completo nAgencia')
print(data.shape[0])
data.head()

merge completo nAgencia
5000


,Agencia_ID,Canal_ID,Cliente_ID,Dev_proxima,Dev_uni_proxima,Producto_ID,Ruta_SAK,Semana,Venta_hoy,Venta_uni_hoy,id,target,tst,Lag1,Lag2,Lag3,Lag4,Lag5,TotalLags,nAgencia
0,1110,7,24014,NaN,NaN,1125,3310,10,NaN,NaN,4627.0,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,0
1,1110,7,57450,NaN,NaN,32393,3310,10,NaN,NaN,1109.0,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,0
2,1110,7,4372181,NaN,NaN,32936,3319,10,NaN,NaN,2549.0,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,0
3,1110,7,2223861,NaN,NaN,1309,3306,11,NaN,NaN,1932.0,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,0
4,1110,11,4702438,NaN,NaN,1146,3504,11,NaN,NaN,2798.0,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,0


In [19]:
nRuta_SAK = pd.DataFrame(data.groupby(['Ruta_SAK','Semana'])['target'].count())
nRuta_SAK = nRuta_SAK.reset_index()
nRuta_SAK.rename(columns={'target': 'nRuta_SAK'}, inplace=True)
nRuta_SAK = pd.DataFrame(nRuta_SAK.groupby(['Ruta_SAK'])['nRuta_SAK'].mean())
nRuta_SAK = nRuta_SAK.reset_index()
 

data = pd.merge(data, nRuta_SAK, 
                            how='left',
                            left_on=['Ruta_SAK'], 
                            right_on=['Ruta_SAK'],
                            left_index=False, right_index=False, sort=True,
                            suffixes=('_x', '_y'), copy=False) 

del nRuta_SAK
gc.collect()
print('merge completo nRuta_SAK')
print(data.shape[0])

nCliente_ID = pd.DataFrame(data.groupby(['Cliente_ID','Semana'])['target'].count())
nCliente_ID = nCliente_ID.reset_index()
nCliente_ID.rename(columns={'target': 'nCliente_ID'}, inplace=True)
nCliente_ID = pd.DataFrame(nCliente_ID.groupby(['Cliente_ID'])['nCliente_ID'].mean())
nCliente_ID = nCliente_ID.reset_index()
 

data = pd.merge(data, nCliente_ID, 
                            how='left',
                            left_on=['Cliente_ID'], 
                            right_on=['Cliente_ID'],
                            left_index=False, right_index=False, sort=True,
                            suffixes=('_x', '_y'), copy=False) 

del nCliente_ID
gc.collect()
print('merge completo nCliente_ID')
print(data.shape[0])

nProducto_ID = pd.DataFrame(data.groupby(['Producto_ID','Semana'])['target'].count())
nProducto_ID = nProducto_ID.reset_index()
nProducto_ID.rename(columns={'target': 'nProducto_ID'}, inplace=True)
nProducto_ID = pd.DataFrame(nProducto_ID.groupby(['Producto_ID'])['nProducto_ID'].mean())
nProducto_ID = nProducto_ID.reset_index()
 

data = pd.merge(data, nProducto_ID, 
                            how='left',
                            left_on=['Producto_ID'], 
                            right_on=['Producto_ID'],
                            left_index=False, right_index=False, sort=True,
                            suffixes=('_x', '_y'), copy=False) 

del nProducto_ID
gc.collect()
print('merge completo nProducto_ID')
print(data.shape[0])


merge completo nRuta_SAK
5000
merge completo nCliente_ID
5000
merge completo nProducto_ID
5000


In [20]:
data.replace(np.nan,0, inplace=True)

train = data[data['tst']==0]
predict = data[data['tst']==1]
train.head()
predict.head()

,Agencia_ID,Canal_ID,Cliente_ID,Dev_proxima,Dev_uni_proxima,Producto_ID,Ruta_SAK,Semana,Venta_hoy,Venta_uni_hoy,...,Lag1,Lag2,Lag3,Lag4,Lag5,TotalLags,nAgencia,nRuta_SAK,nCliente_ID,nProducto_ID
0,1120,1,18707,0.0,0.0,72,1605,11,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0
1,1238,1,104713,0.0,0.0,72,1601,11,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0
2,2040,1,589235,0.0,0.0,72,1110,10,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0
3,1212,1,825934,0.0,0.0,72,1623,11,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0
4,2043,1,1138812,0.0,0.0,72,5022,11,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0


In [21]:
train.head()

,Agencia_ID,Canal_ID,Cliente_ID,Dev_proxima,Dev_uni_proxima,Producto_ID,Ruta_SAK,Semana,Venta_hoy,Venta_uni_hoy,...,Lag1,Lag2,Lag3,Lag4,Lag5,TotalLags,nAgencia,nRuta_SAK,nCliente_ID,nProducto_ID


In [ ]:
i=1
lag = 'Lag' + str(i)
print('Lag:',lag)
data1 = data[['Semana','Cliente_ID','Producto_ID','target']]
data1.head()

In [ ]:
data1.loc[:,'Semana'] = data1['Semana'] +i
data1

In [ ]:
data1 = data1.groupby(['Semana','Cliente_ID','Producto_ID']).mean()
data1.head(n=20)

In [ ]:
data1 = data1.reset_index()
data1.head(n=20)

In [ ]:
data1.rename(columns={'target': lag}, inplace=True)
data1.head()

In [ ]:
data = pd.merge(data,data1,
                how='left',
                left_on=['Semana','Cliente_ID','Producto_ID'], 
                right_on=['Semana','Cliente_ID','Producto_ID'],
                left_index=False, right_index=False, sort=True,
                suffixes=('_x', '_y'), copy=False, )
data.head(n=20)

In [ ]:
data.groupby(['Agencia_ID','Semana']).count().head(n=20)